# Predicting customer churn (attrition)

### By: Raphael Rivers

# Models Building Development 
 

In [1]:
# Import all modules and read in dataset
from modules import*

In [2]:
# retrive standardized numeric data
%store -r Xchurn

In [3]:
# Make a copy of the training data
fit_df = churn_df.copy()

In [4]:
# Apply natural log to non-gaussian variables seen in EDA
fit_df['number_customer_service_calls_log'] = np.log1p(fit_df['number_customer_service_calls'])
fit_df['number_vmail_messages_log'] = np.log1p(fit_df['number_vmail_messages'])
fit_df['total_intl_calls_log'] = np.log1p(fit_df['total_intl_calls'])

In [5]:
# Extract numeric columns and force hard copy
churn_features = fit_df.select_dtypes('number').copy()

In [6]:
# Initialize and fit the scaler
scaler = StandardScaler()
Xchurn1 = scaler.fit_transform(churn_features)

In [7]:
# Convert the returned NumPy array into a DataFrame to support visualizing with Seaborn
Xchurn1_df = pd.DataFrame(Xchurn1, columns=churn_features.columns)

In [8]:
# Perform PCA for dimensionality reduction
pca = PCA(n_components=2)
pca_scores = pca.fit_transform(Xchurn)

# Apply KMeans clustering on the PCA scores
kmeans_pca = KMeans(n_clusters=4, random_state=121)
pca_cluster_labels = kmeans_pca.fit_predict(pca_scores)

# Add PCA cluster labels to the dataframe
fit_df['pca_cluster'] = pca_cluster_labels

# Convert churn to binary
fit_df['churn'] = fit_df['churn'].map({'yes': 1, 'no': 0})

# Select features for the model
features = ['account_length', 'total_day_minutes', 'total_day_calls', 'total_day_charge', 
            'total_eve_minutes', 'total_eve_calls', 'total_eve_charge', 
            'total_night_minutes', 'total_night_calls', 'total_night_charge', 
            'total_intl_minutes', 'total_intl_charge', 
            'number_vmail_messages_log', 'number_customer_service_calls_log', 'total_intl_calls_log',
            'pca_cluster']

# Set model input and output variables
X = fit_df[features]
y = fit_df['churn']

In [9]:
# Store fitted clusters
%store fit_df

Stored 'fit_df' (DataFrame)


In [10]:
# Define the 8 models formulas
formula_list = [
    'churn ~ 1',
    'churn ~ total_day_minutes',
    'churn ~ total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes',
    'churn ~ total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes + pca_cluster',
    'churn ~ pca_cluster * (total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes)',
    'churn ~ (total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes) ** 2',
    'churn ~ total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes + np.power(total_day_minutes, 2) + np.power(total_eve_minutes, 2) + np.power(total_night_minutes, 2) + np.power(total_intl_minutes, 2)',
    'churn ~ pca_cluster + total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes + np.power(total_day_minutes, 2) + np.power(total_eve_minutes, 2) + np.power(total_night_minutes, 2) + np.power(total_intl_minutes, 2)',
    'churn ~ pca_cluster * (total_day_minutes + total_eve_minutes + total_night_minutes + total_intl_minutes + np.power(total_day_minutes, 2) + np.power(total_eve_minutes, 2) + np.power(total_night_minutes, 2) + np.power(total_intl_minutes, 2))'
]

In [11]:
# Store formula list for reuse
%store formula_list

Stored 'formula_list' (list)


In [12]:
# Function to fit and access logistic model
def fit_access_logit(model_name, a_formula, training_data, threshold):
    fit_model = smf.logit(formula=a_formula, data=training_data).fit(disp=False)
    
    training_set_copy = training_data.copy()
    training_set_copy['pred_probability'] = fit_model.predict(training_data)
    training_set_copy['pred_class'] = np.where(training_set_copy['pred_probability'] > threshold, 1, 0)
    
    TN, FP, FN, TP = confusion_matrix(training_set_copy['churn'], training_set_copy['pred_class']).ravel()
    
    Accuracy = (TN + TP) / (TN + FP + FN + TP)
    Sensitivity = TP / (TP + FN)
    Specificity = TN / (TN + FP)
    FPR = 1 - Specificity
    ROC_AUC = roc_auc_score(training_set_copy['churn'], training_set_copy['pred_probability'])
    
    result_dict = {
        'model_name': model_name,
        'model_formula': a_formula,
        'num_coefficients': len(fit_model.params),
        'threshold': threshold,
        'Accuracy': Accuracy,
        'Sensitivity': Sensitivity,
        'Specificity': Specificity,
        'FPR': FPR,
        'ROC_AUC': ROC_AUC
    }
    
    return pd.DataFrame(result_dict, index=[0])

In [13]:
# Initialize result list
result_list = []

# Iterate over the model list and apply the fit function
for m in range(len(formula_list)):
    result_list.append(fit_access_logit(m, formula_list[m], training_data=fit_df, threshold=0.5))

# Concatenate results
results_df = pd.concat(result_list, ignore_index=True)

In [14]:
# View Prediction Result
results_df.sort_values(by='ROC_AUC', ascending=False)

,model_name,model_formula,num_coefficients,threshold,Accuracy,Sensitivity,Specificity,FPR,ROC_AUC
8,8,churn ~ pca_cluster * (total_day_minutes + tot...,18,0.5,0.8752,0.172560,0.990915,0.009085,0.687465
7,7,churn ~ pca_cluster + total_day_minutes + tota...,10,0.5,0.8744,0.162659,0.991614,0.008386,0.675339
6,6,churn ~ total_day_minutes + total_eve_minutes ...,9,0.5,0.8736,0.158416,0.991381,0.008619,0.674831
5,5,churn ~ (total_day_minutes + total_eve_minutes...,11,0.5,0.8718,0.132956,0.993478,0.006522,0.672946
4,4,churn ~ pca_cluster * (total_day_minutes + tot...,10,0.5,0.8624,0.031117,0.999301,0.000699,0.664471
3,3,churn ~ total_day_minutes + total_eve_minutes ...,6,0.5,0.8614,0.024045,0.999301,0.000699,0.659801
2,2,churn ~ total_day_minutes + total_eve_minutes ...,5,0.5,0.8620,0.025460,0.999767,0.000233,0.659236
1,1,churn ~ total_day_minutes,2,0.5,0.8590,0.002829,1.000000,0.000000,0.640135
0,0,churn ~ 1,1,0.5,0.8586,0.000000,1.000000,0.000000,0.500000


### How many coefficient are statistically significant?
To determine the number of statistically significant coefficients, we can fit each logistic regression model and check the p-values of the coefficients. Typically, a p-value less than 0.05 is considered statistically significant.

The number of coefficients estimated for each model is given in the column num_coefficients of the results table. Here is the number of coefficients estimated for each model:

- Model 0: 1 coefficient
- Model 1: 2 coefficients
- Model 2: 5 coefficients
- Model 3: 6 coefficients
- Model 4: 10 coefficients
- Model 5: 11 coefficients
- Model 6: 9 coefficients
- Model 7: 10 coefficients
- Model 8: 18 coefficients

In [15]:
# Define a function to fit the model and extract p-values
def get_significant_coefficients(a_formula, training_data, threshold=0.05):
    fit_model = smf.logit(formula=a_formula, data=training_data).fit()
    p_values = fit_model.pvalues
    significant_coefficients = p_values[p_values < threshold].count()
    return significant_coefficients


In [16]:
# Apply the function to each formula in the formula list
significant_coefficients_list = []

for formula in formula_list:
    significant_coefficients = get_significant_coefficients(formula, fit_df)
    significant_coefficients_list.append(significant_coefficients)

Optimization terminated successfully.
         Current function value: 0.407497
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.385385
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.377481
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.377221
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.376452
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.363993
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.357125
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.356994
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.353014
         Iterations 7


In [17]:
# Create a DataFrame to display the results
sig_coef_df = pd.DataFrame({
    'Model': range(len(formula_list)),
    'Formula': formula_list,
    'Significant Coefficients': significant_coefficients_list
})

# Show significant coeficient result
sig_coef_df

,Model,Formula,Significant Coefficients
0,0,churn ~ 1,1
1,1,churn ~ total_day_minutes,2
2,2,churn ~ total_day_minutes + total_eve_minutes ...,5
3,3,churn ~ total_day_minutes + total_eve_minutes ...,5
4,4,churn ~ pca_cluster * (total_day_minutes + tot...,5
5,5,churn ~ (total_day_minutes + total_eve_minutes...,6
6,6,churn ~ total_day_minutes + total_eve_minutes ...,3
7,7,churn ~ pca_cluster + total_day_minutes + tota...,3
8,8,churn ~ pca_cluster * (total_day_minutes + tot...,10


The number of statistically significant coefficients (features) varies by model, with the highest being 10 significant coefficients in `Models 8` and 6 in `Model 5`. As such which coefficients are statistically significant and whether they are positive or negative, we will need to fit each model and extract the p-values and coefficients.

In [18]:
# Define a function to fit the model and extract significant coefficients along with their signs
def get_significant_coefficients_with_sign(a_formula, training_data, threshold=0.05):
    fit_model = smf.logit(formula=a_formula, data=training_data).fit(disp=False)
    p_values = fit_model.pvalues
    significant_mask = p_values < threshold
    significant_coefficients = p_values[significant_mask].index.tolist()
    coefficient_values = fit_model.params[significant_mask].values
    signs = ['Positive' if coef > 0 else 'Negative' for coef in coefficient_values]
    return list(zip(significant_coefficients, coefficient_values, signs))

In [19]:
# Apply the function to each formula in the formula list
significant_coefficients_details = []

for formula in formula_list:
    details = get_significant_coefficients_with_sign(formula, fit_df)
    significant_coefficients_details.append(details)


In [20]:
# Create a DataFrame to display the results
all_models_result = []

for model_index, details in enumerate(significant_coefficients_details):
    for coef_name, coef_value, sign in details:
        all_models_result.append({
            'Model': model_index,
            'Coefficient': coef_name,
            'Value': coef_value,
            'Sign': sign
        })

sig_coef_sign = pd.DataFrame(all_models_result)


In [21]:
# Show positive and negative coefficient
sig_coef_sign.sort_values(by='Sign', ascending=False)

,Model,Coefficient,Value,Sign
14,4,total_day_minutes,0.010663,Positive
12,3,total_intl_minutes,0.073561,Positive
26,6,"np.power(total_day_minutes, 2)",0.000134,Positive
31,8,total_eve_minutes,0.036447,Positive
23,5,total_day_minutes:total_night_minutes,0.000086,Positive
22,5,total_day_minutes:total_eve_minutes,0.000179,Positive
33,8,"np.power(total_day_minutes, 2)",0.000115,Positive
18,5,Intercept,3.994463,Positive
35,8,"np.power(total_intl_minutes, 2)",0.024732,Positive
16,4,total_intl_minutes,0.104858,Positive


Next let's determine which two models has statistically significant coefficient with the highest magnitude values

In [22]:
# Extract the significant coefficients and their magnitudes
sig_coef_df['Magnitude'] = sig_coef_df['Significant Coefficients'].abs()

# Find the two highest magnitude coefficients
top_two_significant = sig_coef_df.nlargest(2, 'Magnitude')

# Show two coefficients with the highest magnitude
top_two_significant


,Model,Formula,Significant Coefficients,Magnitude
8,8,churn ~ pca_cluster * (total_day_minutes + tot...,10,10
5,5,churn ~ (total_day_minutes + total_eve_minutes...,6,6
